Mục tiêu của các pattern này là: `Don't wait forever`

## Why Timeouts Matter

- Bị treo khi gọi mạng chậm
- Bị block khi service không phản hồi
- Rò rỉ goroutine vì chờ vô hạn
- Trải nghiệm người dùng kém

## Basic Timeout with `time.After`

Đây là timeout pattern sử dụng `select` của Rob Pike:

In [1]:
func basicTimeout() {
    ch := make(chan string)
    
    // Simulate slow operation
    go func() {
        time.Sleep(2 * time.Second)
        ch <- "result"
    }()
    
    select {
    case result := <-ch:
        fmt.Println("Got:", result)
    case <-time.After(1 * time.Second):
        fmt.Println("Timeout!")
    }
}

Ở ví dụ này, chúng ta thấy:
- goroutine được giả lập sleep 2s rồi mới send "result" vào channel
- Có 2 case:
    - result nhận từ ch khi có giá trị truyền vào ch
    - Sau 1s thì timeout
- `select` ở đây:
    - Đang chờ event từ nhiều channels khác nhau
    - Nó sẽ block cho tới khi có channel nào đó ready
    - Thực hiện case đó và thoát
- Sau khi select thoát rồi thì "result" mới được gửi đến ch (do sleep 2s), điều này có thể gây stuck

### Chúng ta xem xét thêm một chút về `time.After` vs `context.WithTimeout`

#### `time.After`

In [2]:
package main

import (
	"fmt"
	"time"
)

func basicTimeoutTimeAfter() {
	fmt.Println("\n--- Case A: time.After ---")
	ch := make(chan string) // unbuffered

	go func() {
		fmt.Println("[worker] started, will sleep 2s...")
		time.Sleep(2 * time.Second)
		fmt.Println("[worker] woke up, trying to send result...")
		ch <- "result" // <- may block forever if nobody receives
		fmt.Println("[worker] send succeeded (you may never see this)")
	}()

	fmt.Println("[main] waiting for result OR timeout (1s)...")
	select {
	case result := <-ch:
		fmt.Println("[main] Got:", result)
	case <-time.After(1 * time.Second):
		fmt.Println("[main] Timeout!")
	}

	fmt.Println("[main] returned from select; main no longer receives from ch.")
	fmt.Println("[main] sleeping 3s to observe worker behavior...")
	time.Sleep(3 * time.Second)
	fmt.Println("[main] done observing.")
}

func main() {
	basicTimeoutTimeAfter()
}



--- Case A: time.After ---
[main] waiting for result OR timeout (1s)...
[worker] started, will sleep 2s...
[main] Timeout!
[main] returned from select; main no longer receives from ch.
[main] sleeping 3s to observe worker behavior...
[worker] woke up, trying to send result...
[main] done observing.


##### Chúng ta thấy là sau khi timeout thì worker vẫn cố gắng gửi đến 1 unbuffered channel, main dừng 3s để quan sát

#### `context.WithTimeout`

In [3]:
package main

import (
	"context"
	"fmt"
	"time"
)

func basicTimeoutWithContext() {
	fmt.Println("\n--- Case B: context.WithTimeout ---")
	ch := make(chan string) // still unbuffered, but safe now

	ctx, cancel := context.WithTimeout(context.Background(), 1*time.Second)
	defer cancel()

	go func() {
		fmt.Println("[worker] started, will sleep 2s...")
		time.Sleep(2 * time.Second)
		fmt.Println("[worker] woke up, wants to send result...")

		select {
		case ch <- "result":
			fmt.Println("[worker] send succeeded")
		case <-ctx.Done():
			fmt.Println("[worker] canceled before send:", ctx.Err())
			return
		}
	}()

	fmt.Println("[main] waiting for result OR ctx timeout (1s)...")
	select {
	case result := <-ch:
		fmt.Println("[main] Got:", result)
	case <-ctx.Done():
		fmt.Println("[main] Timeout/canceled:", ctx.Err())
	}

	fmt.Println("[main] sleeping 3s to observe worker behavior...")
	time.Sleep(3 * time.Second)
	fmt.Println("[main] done observing.")
}

func main() {
	basicTimeoutWithContext()
}



--- Case B: context.WithTimeout ---
[main] waiting for result OR ctx timeout (1s)...
[worker] started, will sleep 2s...
[main] Timeout/canceled: context deadline exceeded
[main] sleeping 3s to observe worker behavior...
[worker] woke up, wants to send result...
[worker] canceled before send: context deadline exceeded
[main] done observing.


##### Ở đây, main timeout sau 1s, sau 2s thì tỉnh dậy để cố send value tới channel, nhưng lúc này `ctx.Done()` đã run, nên sẽ in ra là `canceled before send`

### `time.NewTimer` + Stop()

In [4]:
package main

import (
	"fmt"
	"time"
)

func basicTimeoutWithNewTimer() {
	fmt.Println("\n--- Case C: time.NewTimer + Stop/Drain ---")

	ch := make(chan string)

	go func() {
		fmt.Println("[worker] started, will sleep 2s...")
		time.Sleep(2 * time.Second)
		fmt.Println("[worker] woke up, trying to send result...")

		ch <- "result" // still can block if nobody receives!
		fmt.Println("[worker] send succeeded (you may never see this)")
	}()

	// Create a timer (more controllable than time.After)
	timer := time.NewTimer(1 * time.Second)
	defer func() {
		// Best practice: stop timer + drain if needed to avoid surprises/leaks
		if !timer.Stop() {
			// If Stop() returned false, the timer already fired
			// Drain the channel if there’s a value sitting there
			select {
			case <-timer.C:
			default:
			}
		}
	}()

	fmt.Println("[main] waiting for result OR timer timeout (1s)...")
	select {
	case result := <-ch:
		fmt.Println("[main] Got:", result)

		// We got the result first, so we don't want the timer firing later.
		// (The deferred Stop/Drain will handle it.)
	case <-timer.C:
		fmt.Println("[main] Timeout!")
	}

	fmt.Println("[main] returned from select; main no longer receives from ch.")
	fmt.Println("[main] sleeping 3s to observe worker behavior...")
	time.Sleep(3 * time.Second)
	fmt.Println("[main] done observing.")
}

func main() {
	basicTimeoutWithNewTimer()
}



--- Case C: time.NewTimer + Stop/Drain ---
[main] waiting for result OR timer timeout (1s)...
[worker] started, will sleep 2s...
[main] Timeout!
[main] returned from select; main no longer receives from ch.
[main] sleeping 3s to observe worker behavior...
[worker] woke up, trying to send result...
[main] done observing.


## Timeout Patterns from Rob Pike

### 1. Single Operation Timeout

In [ ]:
func QueryDatabase(query string) (Result, error) {
    resultChan := make(chan Result, 1)
    errChan := make(chan error, 1)
    
    go func() {
        result, err := db.Query(query)
        if err != nil {
            errChan <- err
            return
        }
        resultChan <- result
    }()
    
    select {
    case result := <-resultChan:
        return result, nil
    case err := <-errChan:
        return Result{}, err
    case <-time.After(5 * time.Second):
        return Result{}, fmt.Errorf("query timeout")
    }
}

Đây là pattern mà chạy 1 operation nào đó khá là chậm (query DB chẳng hạn) ở 1 goroutine, và chờ kết quả hoặc eror hoặc timeout. Đây là patten mà: `timeout for waiting, not a timeout for the operation.`

### 2. Whole Conversation Timeout

In [ ]:
func timeoutWholeConversation() {
    c := boring("Joe")
    timeout := time.After(5 * time.Second)
    
    for {
        select {
        case s := <-c:
            fmt.Println(s)
        case <-timeout:
            fmt.Println("You talk too much.")
            return
        }
    }
}

Đây là pattern mà toàn bộ goroutine sẽ timeout trong vòng 5s và return kể từ khi func bắt đầu run.

### 3. Per-Message Timeout

In [ ]:
func perMessageTimeout() {
    c := boring("Joe")
    
    for {
        select {
        case s := <-c:
            fmt.Println(s)
        case <-time.After(1 * time.Second):
            fmt.Println("You're too slow.")
            return
        }
    }
}

Pattern này hơi khác trên chút xíu, `time.After` nằm bên trong loop vô hạn nên sẽ refresh mỗi 1s cho mỗi lần chờ message. Sau khi chờ 1s thì vòng lặp tiếp tục nên sẽ tạo lại timer 1s.

**Cách để nhớ:**
- `Whole conversation timeout` = chờ trong 5s cho toàn bộ
- `Per-message timeout` = mỗi message đến sẽ được refresh lại mỗi 1s

## Advanced Timeout Patterns